***

* [Outline](../0_Introduction/0_introduction.ipynb)
* [Glossary](../0_Introduction/1_glossary.ipynb)
* [2. Mathematical Groundwork](2_0_introduction.ipynb)
    * Previous: [2.8 The Discrete Fourier Transform (DFT) and the Fast Fourier Transform (FFT)](2_8_the_discrete_fourier_transform.ipynb)
    * Next: [2.10 Linear Algrebra](2_10_linear_algebra.ipynb)

***

Import standard modules:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import HTML 
HTML('../style/course.css') #apply general CSS

Import section specific modules:

In [ ]:
pass

In [ ]:
from IPython.display import HTML
HTML('../style/code_toggle.html')

1. [Sampling Theory](#) 
    1. [Nyquist's sampling theorem](#math:sec:nyquists_sampling_theorem)
    2. [Limited sampling](#math:sec:limited_sampling)
    3. [Examples for aliasing and limited sampling](#math:sec:examples_for_aliasing_and_limited_sampling)

## 2.9 Sampling Theory<a id='math:sec:sampling_theory'></a>

When planning physical measurements, the question turns up with which accuracy a measurement has to be made, given the function or the experimental setup. In other words, how finely a function has to be sampled to ideally recover the function: given the diameter of a telescope, or given the maximum separation of two telescopes in an interferometer, what pixel size do we choose in our reconstructed image? With the aid of Fourier theory, we can give an accurate answer to this for a set of functions.

### 2.9.1. Nyquist's Sampling Theorem<a id='math:sec:nyquists_sampling_theorem'></a>

This section describes the concept of sampling, Nyquist rate and aliasing in fundamental signal processing. Consider a real-valued wave package with a cutoff at $\frac{\Delta s}{2}$ ([$\S$ 2.2](2_2_important_functions.ipynb#math:eq:1_003) <!--\ref{math:eq_1_003}-->), which we re-write as a Fourier-transform of a Hermetian function ([$\S$ 2.4](2_4_the_fourier_transform.ipynb#math:sec:fourier_transforms_of_real_valued_and_hermetian_functions)<!--\ref{math:sec:fourier_transforms_of_real_valued_and_hermetian_functions}--> and [$\S$ 2.2](2_2_important_functions.ipynb#math:eq:1_004) <!--\ref{math:eq_1_004}-->) where the Fourier transform has a support $[-\frac{\Delta s}{2},\frac{\Delta s}{2}]$ (with a length of twice the bandwidth).


<a id='math:eq:8_001'></a><!--\label{math:eq:8_001}-->$$
f: \mathbb{R}\rightarrow{R}\\
\begin{align}
f(x)\,&=\,\int_0^{\frac{\Delta s}{2}}A(s)\,\cos{2\pi\imath sx-\phi(s)}\,ds\\
&=\,\int_0^{\frac{\Delta s}{2}}F(s)\,e^{2\pi\imath sx}+F^*(s)\,e^{-2\pi\imath sx}\,ds\\
&=\,\int_0^{\frac{\Delta s}{2}}F(s)\,e^{2\pi\imath sx}+F(-s)\,e^{-2\pi\imath sx}\,ds\\
&=\,\int_{-\frac{\Delta s}{2}}^{\frac{\Delta s}{2}}F(s)\,e^{2\pi\imath sx}\,ds\\
\end{align}
$$

We can hence write

<a id='math:eq:8_002'></a><!--\label{math:eq:8_002}-->$$
\begin{align}
\mathscr{F}\left\{f\right\}(s)\,&=\mathscr{F}\left\{f\right\}(s)\cdot \Pi(\frac{s}{s_0})
\end{align}
$$

Ideally, this function will be "sampled" an infinite number of times with the sampling period $\Delta x$, in an idealised experiment we hence measure the function

<a id='math:eq:8_003'></a><!--\label{math:eq:8_003}-->$$
\begin{align}
f^\prime \,&=\,f(x)\cdot \frac{1}{\Delta x}\,III(\frac{x}{\Delta x})
\end{align}
$$

To judge whether we can recover all "spectral" information about the function, we calculate the Fourier transform (remember the [convolution theorem &#10142;](2_7_fourier_theorems.ipynb#math:sec:convolution_theorem)<!--\ref{math:sec:convolution_theorem}--> and the [Fourier transform of a comb function &#10142;](2_4_the_fourier_transform.ipynb#math:sec:fourier_transform_of_the_comb_function)<!--\ref{math:sec:fourier_transform_of_the_comb_function}-->).

<a id='math:eq:8_004'></a><!--\label{math:eq:8_004}-->$$
\begin{align}
\mathscr{F}\left\{f^\prime\right\}(s)\,&=\,\left(\mathscr{F}\left\{f\right\}\circ III_{\Delta x}\right)(s)\\
&=\,\int_{-\infty}^{\infty} \mathscr{F}\left\{f\right\}(s-t)\,III(\Delta x t)\,dt\\
&=\,\int_{-\infty}^{\infty} \mathscr{F}\left\{f\right\}(s-t)\,\frac{1}{\Delta x}\left(\sum_{n\,=\,{-\infty}}^{+\infty}\delta\left(t-\frac{n}{t}\right)\right)\,dt\\
&=\frac{1}{\Delta x} \sum_{-\infty}^{+\infty}\mathscr{F}\left\{f\right\}\left(s-\frac{n}{\Delta x}\right)\\
(&=\, \frac{1}{\Delta x} \sum_{-\infty}^{+\infty}\mathscr{F}\left\{f\right\}\left(s-\frac{n}{\Delta x}\right)\cdot \Pi\left(\frac{s-\frac{n}{\Delta x}}{s_0}\right) )
\end{align}
$$

The result is an infinite sum of translations of the Fourier transform $\mathscr{F}\left\{f\right\}$ of the original function $f$. The Fourier transform is hence aliased in the Fourier domain. Each summand is offset by $\frac{1}{\delta x}$. Each single repetition has the width of the Fourier transform of the original function $\Delta s$. The inverse Fourier transform would not be reconstructable if there were an overlap of the summands. We can hence conclude:

If we have to sample a function with the bandwidth $\frac{\Delta s}{2}$, the sampling interval has to fulfil the condition

<a id='math:eq:8_005'></a><!--\label{math:eq:8_005}-->$$
\begin{align}
\frac{1}{\Delta x} > \Delta s 
\end{align} \qquad .
$$

This is Nyquist's Sampling Theorem ([<cite data-cite='nyquist'>Harry Nyquist, 1989-1976</cite> &#10548;](https://en.wikipedia.org/wiki/Harry_Nyquist)): to be fully recovered from a measurement, a function with limited bandwidth (in this context also called Nyquist frequency) has to be sampled (infinitely) with a frequency or Nyquist rate of twice the bandwidth (remember that our bandwidth was $\frac{\Delta s}{2}$). Again, the Nyquist rate is twice the Nyquist frequency, for a signal with a bandwidth of 500 Hz has to be sampled at a frequency of 1000 Hz, once every ms. It makes also no sense to sample a funciton at a frequency higher than the Nyquist rate, as we will not gain any more information (than the fully recoverable function).

### 2.9.2. Limited Sampling<a id='math:sec:limited_sampling'></a>

When deriving Nyquist's sampling theorem, we looked at the idealised case of an infinite number of measurements. In practice this is impossible. If we restrict the number of samplings to a finite number N, we measure ($f^{\prime}$ defined in equation [equation](#math:eq:8_003) <!--\ref{math:eq:8_003}-->)

<a id='math:eq:8_006'></a><!--\label{math:eq:8_006}-->$$
\begin{align}
f^{\prime\prime}\,&=\,f^\prime\cdot\sqcap\left(\frac{x-\frac{(N-1)\Delta x}{2}}{(N-1)\Delta x}\right)\\
&=\,\sum_{n\,=\,0}^{N-1}\delta\left(x-n\Delta x\right)f(x)
\end{align} \qquad .
$$

This means that in the Fourier domain we measure ([$\S$ 2.4](2_4_the_fourier_transform.ipynb#math:sec:fourier_transform_of_the_rectangle_and_the_sinc_function)<!--\ref{math:sec:fourier_transform_of_the_rectangle_and_the_sinc_function}-->)

<a id='math:eq:8_007'></a><!--\label{math:eq:8_007}-->$$
\begin{align}
\mathscr{F}\left\{f^{\prime\prime}\right\}(s)\,&=\,\mathscr{F}\left\{f^\prime\right\}\circ \left((N-1)\Delta x\,sinc\left(\left(N-1\right)\Delta x s\right)e^{\mp 2\pi\frac{(N-1)\Delta x}{2}}\right)\\
\end{align} \qquad ,
$$

which, in other words, means that our measurement results in an aliased measurement of the Fourier transform *convolved with a sinc function*, which gets narrower with an increased number of measurements. We hence know that we cannot recover the real Fourier transform (and with that the original function) with infinite accuracy. Notice that the discrete Fourier transform will determine the Fourier transform just at the Nyquist rate $\frac{1}{N\Delta x}$, which is determined by the box function.

### 2.9.3. Examples of Aliasing and Nyquist's Sampling Theorem<a id='math:sec:examples_for_aliasing_and_limited_sampling'></a>

We again address the questions:

1. What is sampling and  does sampling do?
2. What is the best criteria for perfect sampling?
3. Does ideal sampling exists in pratice?

Basically, sampling is the process  by which continuous  time signals or analog signal (such as voices)  are turned into discrete time signals. We sample a continuous time variant signal so that we could manage to save and process the signal into computers. To reconstruct the continuous time signal,  we apply the Fourier transform to the sampled signal and apply filters theories in other to bandlimit the results of the Fourier transform.

Before we start the analysis of sampling, the following definitions are required.

1. We will refer to the continuous time variant signal as $\textbf{physical signal}$.
2. The descrete signal as $\textbf{sampled signal}$.

The process of sampling is easy to represent mathematically. Suppose a physical signal $d_r (x)$ and $d_s (x)$ its sampled form i.e the descrete signal, and $\Delta x$ the sampled interval. According to Shannon the sampled signal is expressed as:

$$
\begin{alignat}{2}
d_s(x) &= s(x) d_r(x),~~~~ Eq.1
\end{alignat}
$$

where  $s(x)=\sum_{k=-\infty}^{+\infty}\delta(x-k\Delta x)$, is an infinite series of Delta functions known as a Dirac Comb at a regular spacing $\Delta x$ and  $k$ is the number of  discrete points on the sampling grid. 


To see how this happens pratically, consider a sine waveform $d_r=sin(2\pi f n x +cst)$ at a frequency $f=4 Hz$ with a total number of samples $n=100$. Note here that the maximum frequency present in this sine wave is $f_{max}=nf$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

f = 4;   # frequency of the physical signal
plt.axhline(y=0,xmin=0,xmax=250,color='r')
plt.plot(np.linspace(0,1,100.),np.sin(2*np.pi*f*np.linspace(0,1,100.)), 'k')

plt.show()

Now supose that this  sine wave is the physical signal i.e the continuous time variant signal. If we sample this signal with sampling intervals of  $\Delta x=1/80$, $\Delta x=1/50$ and $\Delta x=1/25$ then we have the following.

In [ ]:
y = np.sin(2*np.pi*f*np.linspace(0,1,80.))
plt.stem(np.linspace(0,1,80.),y,'-')
plt.ylabel("Amplitude")
plt.title("sampled signal with sampling interval $1/80$")
plt.show()


y = np.sin(2*np.pi*f*np.linspace(0,1,50.))
plt.stem(np.linspace(0,1,50.),y,'-')
plt.ylabel("Amplitude")
plt.title("sampled signal with sampling interval $1/50$")
plt.show()

y = np.sin(2*np.pi*f*np.linspace(0,1,25.))
plt.stem(np.linspace(0,1,25.),y,'-')
plt.ylabel("Amplitude")
plt.title("sampled signal with sampling interval $1/20$")
plt.show()

There exit between the samples empty gaps. These emplty gaps are points where the physical signal was not sampled. Keep know in mind that these empty gaps represent all the informations lost during the sampling process. Note also that, in the case where the sampling interval is $\Delta x=\frac{1}{20}$, we can not spot all the peaks in this sine wave. 

In general, to preserve all the physical signal peaks, it is necessary to sample at least twice the maximum frequency present in the physical signal. In our case, the maximum frequency of the sine wave is $f_{max}=100\times 4$, therefore, any sampling interval $\Delta x$ above $\frac{1}{2f_{max}}$ will not exactly reconstruct the physical signal. 

***

<span style="background-color:cyan">Below this line this section is incomplete</span>

# Nyquist theorem

The Nyquist theorem state that the sampling frequency  must be greater than twice the 
highest frequency contained in the signal i.e $\Delta x\leq \frac{1}{2f_{max}}$. The $2f_{max}$ is known as the  Nyquist rate and any sampling bellow the Nyquist rate introduces sampling artefacts in the recontructed signal. These sampling artefacts are known in signal processing as $\textbf{aliasing}$.

# Aliasing and demonstration

Let us sopose we want to sample a Gaussian waveform $d_r (x) = e^{\frac{-(x-\mu)^2}{2\sigma^2}}$

In [ ]:

def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

x = np.linspace(-50,50,50)
mu = 0.
sig = 10.
g = gaussian(x,mu,sig)

fig = plt.figure()
fig.subplots_adjust(hspace=.9)

ax1 = fig.add_subplot(311)
ax1.plot(x,g)
plt.ylim(-0.5,1.5)
#ax1.axhline(y=0., xmin=-30., xmax=50., color = 'r')
plt.title("Physical signal: continuous signal")


## delta dirac comb parameter
step = 15 # sampling step
deltacom = np.ones(step)
x1 = np.linspace(-50,50,step)
ax2 = fig.add_subplot(312)
markerline, stemlines, baseline = ax2.stem(x1, deltacom, '-')
plt.title("Delta Dirac Comb: sampling function")
plt.ylim(-0.5,1.5)


## sampled Gaussian
ax3 = fig.add_subplot(313)
markerline, stemlines, baseline = ax3.stem(x1, gaussian(x1,mu,sig), '-')
plt.ylim(-0.5,1.5)
plt.title("Sampled signal")
plt.show()


The physical signal (top plot) is multiplied by the Dirac Comb function (middle plot) and this result to the observed signal (bottom centre).

We can write each discrete point on the sampled signal as 
$$
\begin{alignat}{2}
d_s(x)_k &= d_r(k\Delta x) ~~~~ Eq.2
\end{alignat}
$$

Note from the example that, $\Delta x = 15$ is the sampling interval or the sampling step.

Combining Eq.1 and Eq.2, the sampled signal becomes the sum of each discrete 
point on the sample grid multiplied by a  Dirac delta function. 

$$
\begin{alignat}{2}
d_s(x) &= \sum_{k=-\infty}^{+\infty}\delta(x)d_s(x)_k. ~~~~ Eq.3
\end{alignat}
$$

Since we are interested in the Fourier  and the time-domain signal, the Fourier transform is applied to Eq.1. According to the convolution theorem which states that the Fourier transform of the product of two functions  yields to the convolution of their respective Fourier transform. Eq.1 becomes 

$$
\begin{alignat}{2}
\mathcal{F} \{d_s \}(y)  &=\Big(\mathcal{F} \big\{s\big\}\circ \mathcal{F} \big\{d_r\big\}\Big)(y),~~~~ Eq.4
\end{alignat}
$$

where $\mathcal{F}^{}$ denotes the Fourier transform and $\circ$ is the convolution operation. 
The Fourier transform of a Dirac Comb of regular spacing $\Delta x$, is a Dirac Comb of regular spacing $\Delta y$

$$
\begin{alignat}{2}
\mathcal{F}\big\{s\big\} & =\Delta y\sum_{k=-\infty}^{+\infty}\delta(y-k\Delta y),~~~~ Eq.5
\end{alignat}
$$

where $\Delta y= 2\pi/ \Delta x$  is the sampling rate. Combining Eq.4 and Eq.5 we obtain the following

$$
\begin{alignat}{2}
\mathcal{F}^{}\big\{d_s\big\}(y) &=\Big(\Delta y \sum_{k=-\infty}^{+\infty}\delta(y-k\Delta y)\Big)\circ\Big(\mathcal{F}\{d_r\}(y)\Big)\\
				  &=\Delta y \sum_{k=-\infty}^{+\infty}\mathcal{F}\{d_r\}(y-k\Delta y).~~~~ Eq.6
\end{alignat}
$$

See Appendix A for the detailed proof of Eq.5 and Eq.6.

For completeness, the spectrum from the point $k=0$ in Eq.6 can be separated 
\begin{alignat}{2}
\mathcal{F}^{}\big\{d_s\big\}(y) &= \Delta y\mathcal{F}^{}\big\{d_r\big\}(y) + \Delta y\sum_{k=-\infty, k\neq 0}^{+\infty}\mathcal{F}^{}\big\{d_r\big\}(y-k\Delta y)\\
				&\simeq \mathcal{F}^{}\big\{d_r\big\}(y) + \sum_{k=-\infty, k\neq0}^{+\infty}\mathcal{F}^{}\big\{d_r\big\}(y-k\Delta y) ~~~~ Eq.7
\end{alignat}


It is shown in Eq.7 that:


 1. The Fourier transform of the sampled signal duplicates the Fourier transform 
 of the physical signal at each point in the Fourier transformed of the Dirac Comb. 
 
 2. Sampling and the Fourier transform conserves the physical signal $\mathcal{F}^{}\big\{d_r\big\}(y)$ at the point $k=0$  and replicates it at all points $k\neq0$ with regular spacing $\Delta y$.
 
 3. The copies for which $k\neq0$ i.e, $\sum_{k=-\infty, k\neq0}^{+\infty}\mathcal{F}^{}\big\{d_r\big\}(y-k\Delta y)$ are aliased spectra.
 
 These remarks are illustrated on the following:


In [ ]:
fft_gau = np.fft.fftshift(np.abs(np.fft.fft(g)))
fft_gau = fft_gau/fft_gau.max()


fig = plt.figure()
fig.subplots_adjust(hspace=.9)


ax1 = fig.add_subplot(311)
ax1.plot(x,fft_gau)
plt.ylim(-0.5,fft_gau.max())
plt.title("Fourrier transform of the Physical signal: Gaussian")


## delta dirac comb parameter
nb_sample = 5
deltacom = np.ones(nb_sample)
x2 = np.linspace(-50,50,nb_sample)
ax2 = fig.add_subplot(312)
markerline, stemlines, baseline = ax2.stem(x2, deltacom, '-')
plt.title("Delta Dirac Comb: Fourier transform of the sampling function")
plt.ylim(-0.5,1.5)


## sampled Gaussian
ax3 = fig.add_subplot(313)
ax3.plot(x-0,fft_gau,'b')
ax3.plot(x-25.,fft_gau, 'k')
ax3.plot(x+25,fft_gau, 'k')
ax3.plot(x-50.,fft_gau, 'k')
ax3.plot(x+50,fft_gau, 'k')
plt.ylim(-0.5,fft_gau.max())
plt.title("Fourrier tranform of the Sampled signal")
plt.show()


The Fourier transform of the physical signal (Top plot). The Fourier transform of the Dirac Com (centre plot). The Fourier transform of the observed signal (bottoom plot).

The plot shows that, the Fourier tranform of the observed signal is a supperposition of the Fourier tranform of the physical signal at each point in the Dirac Comb. At the point $k=0$ (the blue curve) is the Fourier transform of the physical signal, and all other copies at point $k\neq 0$ are aliasing copies. Which shows that any reconstruction of $d_r$ from $d_s$ requires the suppression of the aliased spectra i.e all copies of $\mathcal{F}^{}\big\{d_r\big\}$ for which $k\neq 0$.



A method to suppress the aliased copies is to multiply $\mathcal{F}^{}\big\{d_s\big\}$, by a tophat window, as a tophat window is unity in the pass-band (area of which $k=0$) and zero in the stop-band (area of which $k\neq 0$).

Note that, a complete reconstruction of $d_r$ requires the absence of overlapping copies of the aliased spectra with the Fourier transform of the physical signal i.e $\mathcal{F}^{}\big\{d_r\big\}$ for which $k=0$. Otherwise, $d_r$ cannot be uniquely reconstructed. The following conditions must hold for a unique recovery:

1. The observed signal must be band-limited in order to avoid that $\mathcal{F}^{}\big\{d_r\big\}(y)$ at point $k=0$ extends infinitely and overlaps with its aliasing copies.
2. The sampling frequency $\Delta y$ must be greater than twice the highest frequency $f_s$ contained in the signal ($\Delta y\geq 2 f_s$). The quantity $2 f_s$ is the minimum sampling frequency or the  Nyquist rate  required to avoids aliasing.

We demonstrate this in the following example:


In [ ]:
## sampled Gaussian
plt.plot(x-0,fft_gau,'b')
plt.plot(x-10.,fft_gau, 'k')
plt.plot(x+10,fft_gau, 'k')
plt.ylim(-0.1,fft_gau.max())
plt.title("Fourrier tranform of the Sampled signal: aliasing")
plt.show()



Let us now supose that these conditions are satisfied and that  $\Pi^{}$ is a tophat window, given by

$$
\begin{equation}
\Pi^{}(y) = \bigg \{ \begin{array}{cl}
1, &  k=0 \\
0, & \mathrm{otherwise}
\end{array}
\end{equation}
$$

Let us introduce the observed signal $d_o$ and multiply Eq.7 by $\Pi^{}$. 

$$
\begin{alignat}{2}
\mathcal{F}\big\{d_o\big\}(y) &=\Pi^{}(y)\mathcal{F}^{}\big\{d_s\big\}(y),~~~~ Eq.8
\end{alignat}
$$

which will lead to

$$
\begin{alignat}{2}
d_o(x) &= \Bigg(\mathcal{F}^{-1}\{ \Pi^{}\}\circ d_s\Bigg)(x)~~~~ Eq.9
\end{alignat}
$$

In [ ]:
fft_gau = np.fft.fftshift(np.abs(np.fft.fft(g)))
fft_gau = fft_gau/fft_gau.max()


fig = plt.figure()
fig.subplots_adjust(hspace=.9)


## sampled Gaussian
ax3 = fig.add_subplot(311)
ax3.plot(x-0,fft_gau,'b')
ax3.plot(x-25.,fft_gau, 'k')
ax3.plot(x+25,fft_gau, 'k')
ax3.plot(x-50.,fft_gau, 'k')
ax3.plot(x+50,fft_gau, 'k')
plt.ylim(-0.5,fft_gau.max())
plt.title("Fourrier tranform of the Sampled signal")


ax1 = fig.add_subplot(312)
boxcar = np.zeros(len(x))
boxcar[len(x)/2-2:len(x)/2+2]=1.
ax1.plot(x,boxcar)
plt.ylim(-0.5,1.1)
plt.title("Fourrier transform of the Physical signal: Gaussian")


## delta dirac comb parameter
ax3 = fig.add_subplot(313)
ax3.plot(x-0,fft_gau,'b')
plt.ylim(-0.5,fft_gau.max())
plt.title("Fourrier tranform of the Sampled signal")

plt.show()


Since there is no overlapping aliasing spectrum with $\mathcal{F}^{}\big\{d_r\big\}(y)$ at the point $k=0$, one can still write Eq.8 as follows

$$
\begin{alignat}{2}
\mathcal{F}\big\{d_o\big\}(y)&=\Pi^{[k]}(y)\mathcal{F}^{}\big\{d_s\big\}(y)\\
	      &\simeq \Pi^{[k]}(y)\Bigg(\mathcal{F}^{}\big\{d_r\big\}(y) + \sum_{k=-\infty, k\neq0}^{+\infty}\mathcal{F}^{}\big\{d_r\big\}(y-k\Delta y)\Bigg)\\
	    &\simeq\Pi^{[k]}(y)\mathcal{F}^{}\big\{d_r\big\}(y).~~~~ Eq.10
\end{alignat}
$$


Note from the subtitution that $\Pi^{[k]}(y)\sum_{k=-\infty, k\neq0}^{+\infty}\mathcal{F}^{}\big\{d_r\big\}(y-k\Delta y)=0$. We can
rewrite Eq.10 as a convolution

$$
\begin{alignat}{2}
d_o(x) &\simeq\Bigg(\mathcal{F}^{-1}\{ \Pi^{[k]}\}\circ d_r\Bigg)(x).~~~~ Eq.11
\end{alignat}
$$

Solving  Eq.9 equal to Eq.11 we have

$$
\begin{alignat}{2}
d_s&\simeq d_r.
\end{alignat}
$$

The physical signal can be reconstructed from the observed signal if all conditions are satisfied. However, ideal sampling can not be applicable in real life problems. This is due to the convolution of the physical signal with a low pass taper which is the sinc filter ($\mathcal{F}^{-1}\{ \Pi^{}\}$) and thereby could only be represented in finite support.


***

* Next: [2.10 Linear Algrebra](2_10_linear_algebra.ipynb)